In [13]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()
# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [17]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import warnings
import getpass
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode

In [18]:
class DocumentRetriever:
    def __init__(self, persist_directory):
        self.persist_directory = persist_directory
        self.embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={'device': 'mps'})
        self.vector_db = Chroma(persist_directory=self.persist_directory, embedding_function=self.embeddings)

    def retrieve_documents(self, query, top_k=5):
        # Retrieve top_k relevant document chunks based on the query
        return self.vector_db.similarity_search(query, k=top_k)

In [19]:
persist_directory = './chroma_db'
retriever = DocumentRetriever(persist_directory=persist_directory)
vector_store = retriever.vector_db

In [2]:

# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
# os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [3]:
# from langchain.vectorstores import Chroma
# vector_store = Chroma(embedding_function=embeddings)



/var/folders/nl/g_2pwbq97qnbzklz6nttyzqr0000gn/T/ipykernel_67571/1368482891.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(embedding_function=embeddings)


In [4]:
# import bs4
# from langchain import hub
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_core.documents import Document
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from typing_extensions import List, TypedDict

# # Load and chunk contents of the blog
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# all_splits = text_splitter.split_documents(docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# # Index chunks
# _ = vector_store.add_documents(documents=all_splits)

In [20]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)


In [21]:
from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: Hello\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [22]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an Medical assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [11]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [14]:
input_message = "Hello"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hello
================================== Ai Message ==================================

Hello! How can I assist you today?


In [31]:
input_message = "Can you tell me if a patient with cancer should undergo inpatient or outpatient surgery?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Can you tell me if a patient with cancer should undergo inpatient or outpatient surgery?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_H9B3vRFINwFEK3mkSDgk6ROX)
 Call ID: call_H9B3vRFINwFEK3mkSDgk6ROX
  Args:
    query: inpatient vs outpatient surgery for cancer patients
================================= Tool Message =================================
Name: retrieve

Source: Hello
Content: General Theme – What determines if an outpatient surgery should be done at KOP versus main OR? This is not always black and white and will require clinical judgement. The following guidelines aim to set some expectations but does not fill all the holes. In general, the thought process is that if the MD feels the following, surgery may need to move to the main hospital:
patient will be very high risk for admission, patient should be done in the main operating

In [16]:
input_message = "Can you look up some common ways of doing it?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up some common ways of doing it?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_xs89WtZCfhhtqdWcKMvtcPAN)
 Call ID: call_xs89WtZCfhhtqdWcKMvtcPAN
  Args:
    query: common ways of doing it
================================= Tool Message =================================
Name: retrieve

Source: Hello
Content: within a relatively short period. Patients with multi-
ple comorbidities, or who are frail, severely depressed
or have poor nutritional status may have the most to
gain from a multimodal prehabilitation program
[18]. Typical prehabilitation programs tend to use a
multimodal approach incorporating two or more
interventions. Patient and family education allows
for clear expectation setting, encourages patient par-
ticipation and engagement in their recovery, reduces
anxiety, and increases patient satisfaction [19]. Most
studies o